<h1>POwB - QIIME2</h1>
<h2>Kaja Łucka, Daryna Yakymenko, Klaudia Jamicka </h2> 

https://docs.qiime2.org/2024.10/tutorials/moving-pictures/

<h3>0. Przygotowanie</h3>

Przygotowuję środowisko i instaluję QIIME 2 2024.10 (Amplicon Distribution)

<p style="color: red;">plik <i>powb-qiime.yml</i> w folderze envs</p>

In [ ]:
%%bash
conda update conda
conda env create -n powb-qiime --file https://data.qiime2.org/distro/amplicon/qiime2-amplicon-2024.10-py310-linux-conda.yml
conda activate powb-qiime
qiime --help 

Tworzę osobny katalog i do niego przechodzę

<p style="color: red;">zbędne w potoku</p>

In [ ]:
%%bash
mkdir qiime2-moving-pictures-tutorial
cd qiime2-moving-pictures-tutorial

<h3>1. Metadane </h3>

Pobieram zaproponowane przez tutorial metadane
<i><p style="color: red;">process download_data</p></i>

In [ ]:
%%bash
wget \
  -O "sample-metadata.tsv" \
  "https://data.qiime2.org/2024.10/tutorials/moving-pictures/sample_metadata.tsv"

<h3>2. Dane </h3>

Pobieram zaproponowane przez tutorial dane

<i><p style="color: red;">process download_data</p></i>

In [ ]:
%%bash
mkdir emp-single-end-sequences

wget \
  -O "emp-single-end-sequences/barcodes.fastq.gz" \
  "https://data.qiime2.org/2024.10/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"

wget \
  -O "emp-single-end-sequences/sequences.fastq.gz" \
  "https://data.qiime2.org/2024.10/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"

Importuję dane do QIIME
<i><p style="color: red;">process import_sequences_to_qiime</p></i>

In [ ]:
%%bash
qiime tools import \
  --type EMPSingleEndSequences \
  --input-path emp-single-end-sequences \
  --output-path emp-single-end-sequences.qza

Sprawdzam, czy się dobrze zaimportowało
<p style="color: red;">zbędne w potoku</p>

In [ ]:
%%bash
qiime tools peek emp-single-end-sequences.qza

<h3>3. Demultipleksowanie </h3>

<i><p style="color: red;">process demultiplexing</p></i>

In [ ]:
%%bash
qiime demux emp-single \
  --i-seqs emp-single-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --o-per-sample-sequences demux.qza \
  --o-error-correction-details demux-details.qza

Generuję raport

<i><p style="color: red;">process demultiplexing_report</p></i>

In [ ]:
%%bash
qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

qiime tools view demux.qzv

<h3>4. QC sekwencji i tworzenie feature table </h3>

<h4>Opcja 1: DADA 2 </h4>
<p style="color: red;">Opcja domyślna</p> 

<i><p style="color: red;">process qc_dada2</p></i>

In [ ]:
%%bash
qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 120 \
  --o-representative-sequences rep-seqs-dada2.qza \
  --o-table table-dada2.qza \
  --o-denoising-stats stats-dada2.qza

Wizualizacja

<i><p style="color: red;">process qc_dada2_vis</p></i>

In [ ]:
%%bash
qiime metadata tabulate \
  --m-input-file stats-dada2.qza \
  --o-visualization stats-dada2.qzv

qiime tools view stats-dada2.qzv

Jeśli chcemy użyć tej tabeli w dalszej części workflow, korzystamy z tych komend

<p style="color: red;">zbędne w potoku</p>

In [ ]:
%%bash
mv rep-seqs-dada2.qza rep-seqs.qza
mv table-dada2.qza table.qza

<h4>Opcja 2: Deblur </h4>

<p style="color: red;">nextflow run main.nf -with-conda --qc "deblur"</p> 

<i><p style="color: red;">process qc_deblur_step1</p></i>

In [ ]:
%%bash
qiime quality-filter q-score \
 --i-demux demux.qza \
 --o-filtered-sequences demux-filtered.qza \
 --o-filter-stats demux-filter-stats.qza



<i><p style="color: red;">process qc_deblur_step2</p></i>

In [ ]:
%%bash
qiime deblur denoise-16S \
  --i-demultiplexed-seqs demux-filtered.qza \
  --p-trim-length 120 \
  --o-representative-sequences rep-seqs-deblur.qza \
  --o-table table-deblur.qza \
  --p-sample-stats \
  --o-stats deblur-stats.qza

Wizualizacje


<i><p style="color: red;">process qc_deblur_vis</p></i>

In [ ]:
%%bash
qiime metadata tabulate \
  --m-input-file demux-filter-stats.qza \
  --o-visualization demux-filter-stats.qzv
qiime deblur visualize-stats \
  --i-deblur-stats deblur-stats.qza \
  --o-visualization deblur-stats.qzv

qiime tools view demux-filter-stats.qzv
qiime tools view deblur-stats.qzv

Jeśli chcemy użyć tej tabeli w dalszej części workflow, korzystamy z tych komend

<p style="color: red;">zbędne w potoku</p>

In [ ]:
mv rep-seqs-deblur.qza rep-seqs.qza
mv table-deblur.qza table.qza